In [1]:
# Import packages
import pathlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import pandas as pd
from skimage import exposure
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0 #MODEL B
import time

### Load in the models

In [2]:
model_garden = tf.keras.models.load_model('..\\saved_model\\model_garden_unaugmented')
model_king = tf.keras.models.load_model('..\\saved_model\\model_kingfisher_unaugmented')
model_raptor = tf.keras.models.load_model('..\\saved_model\\model_raptor_unaugmented')
model_water = tf.keras.models.load_model('..\\saved_model\\model_waterbird_unaugmented')
model_wood = tf.keras.models.load_model('..\\saved_model\\model_woodpecker_unaugmented')

### Define classes for each model

In [3]:
CLASS_GARDEN = ['Ashy Tailorbird',
 'Asian Glossy Starling',
 'Asian Koel',
 'Black-naped Oriole',
 'Blue-tailed Bee-eater',
 'Blue-throated Bee-eater',
 'Brown Shrike',
 'Brown-throated Sunbird',
 'Common Iora',
 'Common Myna',
 'Common Tailorbird',
 'Javan Myna',
 'Malaysian Pied Fantail',
 'Olive-backed Sunbird',
 'Oriental Magpie-Robin',
 'Oriental Pied Hornbill',
 'Pacific Swallow',
 'Pink-necked Green Pigeon',
 'Rose-ringed Parakeet',
 'Savanna Nightjar',
 'Scarlet-backed Flowerpecker',
 'Yellow-vented Bulbul',
 'Zebra Dove']
CLASS_KING = ['Collared Kingfisher',
 'Common Kingfisher',
 'Stork-billed Kingfisher',
 'White-throated Kingfisher']
CLASS_RAPTOR = ['Brahminy kite',
 'Buffy fish owl',
 'Crested serpent eagle',
 'Grey-headed fish eagle',
 'Spotted wood owl',
 'White-bellied sea eagle']
CLASS_WATER = ['Black-crowned night heron',
 'Chinese pond heron',
 'Common sandpiper',
 'Eastern cattle egret',
 'Grey heron',
 'Little egret',
 'Purple heron',
 'Striated heron',
 'White-breasted waterhen',
 'Yellow bittern']
CLASS_WOOD = ['Common Flameback',
 'Coppersmith Barbet',
 'Laced Woodpecker',
 'Rufous Woodpecker',
 'Sunda Pygmy Woodpecker']

In [4]:
NUM_GARDEN = len(CLASS_GARDEN)
NUM_KING = len(CLASS_KING)
NUM_RAPTOR = len(CLASS_RAPTOR)
NUM_WATER = len(CLASS_WATER)
NUM_WOOD = len(CLASS_WOOD)

### Prediction functions

In [5]:
IMG_SIZE = 224

In [6]:
# Function to process and predict image
def image_prediction(image, MODEL, CLASS_NAMES, NUM_CLASSES):
    # Process input image
    image_resized = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image_resized, axis=0)
    
    # Threshold
    threshold = (1/NUM_CLASSES) * 1.2 # 120% from uniform split probability as threshold
    
    # Run image through model to obtain prediction
    pred = MODEL.predict(image)
        # pred is a array of length class_count. Each entry is the probability for a particular class
        # Index based on class_names
        
    if pred[0][np.argmax(pred)] < threshold:
        pred_class = "Unknown"
        sig_prob = 0
        
    else:    
        # Obtain prediction class
        pred_class = CLASS_NAMES[np.argmax(pred)]
        
        uniform_prob = (1/NUM_CLASSES)
        max_prob = pred[0][np.argmax(pred)]
        sig_prob = (max_prob-uniform_prob)/uniform_prob
        
    return sig_prob, pred_class

In [9]:
def ensemble_prediction (image):
    garden_prob, garden_pred = image_prediction(image, model_garden, CLASS_GARDEN, NUM_GARDEN)
    king_prob, king_pred = image_prediction(image, model_king, CLASS_KING, NUM_KING)
    raptor_prob, raptor_pred = image_prediction(image, model_raptor, CLASS_RAPTOR, NUM_RAPTOR)
    water_prob, water_pred = image_prediction(image, model_water, CLASS_WATER, NUM_WATER)
    wood_prob, wood_pred = image_prediction(image, model_wood, CLASS_WOOD, NUM_WOOD)
    
    ensemble_prob = [garden_prob, king_prob, raptor_prob, water_prob, wood_prob]
    ensemble_pred = [garden_pred, king_pred, raptor_pred, water_pred, wood_pred]
    
    return ensemble_pred[np.argmax(ensemble_prob)]

### Ensemble Architecture

Evaluation

In [15]:
# Test directory with all 48 classes
test_dir = pathlib.Path('C:/Users/hanis/OneDrive - National University of Singapore/ML PROJ/processed_data_2/data(48_class)/test')

In [7]:
ALL_CLASS_NAMES = ['Ashy Tailorbird',
 'Asian Glossy Starling',
 'Asian Koel',
 'Black-crowned night heron',
 'Black-naped Oriole',
 'Blue-tailed Bee-eater',
 'Blue-throated Bee-eater',
 'Brahminy kite',
 'Brown Shrike',
 'Brown-throated Sunbird',
 'Buffy fish owl',
 'Chinese pond heron',
 'Collared Kingfisher',
 'Common Flameback',
 'Common Iora',
 'Common Kingfisher',
 'Common Myna',
 'Common Tailorbird',
 'Common sandpiper',
 'Coppersmith Barbet',
 'Crested serpent eagle',
 'Eastern cattle egret',
 'Grey heron',
 'Grey-headed fish eagle',
 'Javan Myna',
 'Laced Woodpecker',
 'Little egret',
 'Malaysian Pied Fantail',
 'Olive-backed Sunbird',
 'Oriental Magpie-Robin',
 'Oriental Pied Hornbill',
 'Pacific Swallow',
 'Pink-necked Green Pigeon',
 'Purple heron',
 'Rose-ringed Parakeet',
 'Rufous Woodpecker',
 'Savanna Nightjar',
 'Scarlet-backed Flowerpecker',
 'Spotted wood owl',
 'Stork-billed Kingfisher',
 'Striated heron',
 'Sunda Pygmy Woodpecker',
 'White-bellied sea eagle',
 'White-breasted waterhen',
 'White-throated Kingfisher',
 'Yellow bittern',
 'Yellow-vented Bulbul',
 'Zebra Dove']

TOTAL_NUM_CLASS = len(ALL_CLASS_NAMES)

In [17]:
total_count   = [0] * TOTAL_NUM_CLASS
true_positive = [0] * TOTAL_NUM_CLASS
false_negative = [0] * TOTAL_NUM_CLASS
false_positive = [0] * TOTAL_NUM_CLASS
unknown_count = 0

for bird_folder in test_dir.rglob("*"):
    for bird_image in bird_folder.rglob("*.jpg"):
        
        bird_species = str(bird_folder.stem)
        bird_index = ALL_CLASS_NAMES.index(bird_species)
        
        # Update total count
        total_count[bird_index] += 1
        
        # Load image
        image = cv2.imread(str(bird_image))
        
        # Predict classification
        pred_class = ensemble_prediction(image)
        
        if str(pred_class) == bird_species:
            true_positive[bird_index] += 1
            
        elif str(pred_class) == "Unknown":
            unknown_count += 1
            
        else:
            false_negative[bird_index] += 1  
            false_positive[ALL_CLASS_NAMES.index(pred_class)] += 1

In [19]:
unknown_count

0

In [19]:
results = pd.DataFrame({'Class': ALL_CLASS_NAMES,
                        'total': total_count,
                        'true_positive' : true_positive,
                        'false_negative' : false_negative,
                        'false_positive' : false_positive})

In [15]:
results['Precision'] = results['true_positive']/(results['true_positive']+results['false_positive'])
results['Recall'] = results['true_positive']/(results['true_positive']+results['false_negative'])
results['F1 Score'] = (2/((1/results['Precision'])+(1/results['Recall'])))

In [16]:
results

,Class,total,true_positive,false_negative,false_positive,Precision,Recall,F1 Score
0,Ashy Tailorbird,9,4,5,5,0.444444,0.444444,0.444444
1,Asian Glossy Starling,8,6,2,35,0.146341,0.750000,0.244898
2,Asian Koel,7,2,5,29,0.064516,0.285714,0.105263
3,Black-crowned night heron,8,2,6,4,0.333333,0.250000,0.285714
4,Black-naped Oriole,7,0,7,6,0.000000,0.000000,0.000000
5,Blue-tailed Bee-eater,8,3,5,16,0.157895,0.375000,0.222222
6,Blue-throated Bee-eater,10,8,2,57,0.123077,0.800000,0.213333
7,Brahminy kite,8,0,8,0,NaN,0.000000,NaN
8,Brown Shrike,8,7,1,15,0.318182,0.875000,0.466667
9,Brown-throated Sunbird,10,4,6,8,0.333333,0.400000,0.363636


In [17]:
filepath = pathlib.Path('C:/Users/hanis/OneDrive - National University of Singapore/ML PROJ/results_csv/resnet50_modified.csv') 
results.to_csv(filepath, index = False) 

In [21]:
test_image_dir = pathlib.Path('..\\data\\bird.jpg')

test_image = cv2.imread(str(test_image_dir))

tic = time.perf_counter()

test_pred_class = ensemble_prediction(test_image)

t = time.perf_counter() - tic
print(t, ' seconds')
print(test_pred_class)

0.34655750000001717  seconds
Asian Glossy Starling
